## Домашнее задание №8

В данном задании вам предстоит детально рассмотреть механизм Attention (и реализовать несколько его вариантов), а также вернуться к задаче классификации текстов из задания №6 и решить ее с использованием BERT.

In [1]:
import os
import random

import numpy as np
import pandas as pd

from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score

import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
from IPython.display import clear_output 
%matplotlib inline

### Задание №2 (опциональное). Классификация текстов с использованием предобученной языковой модели.

Вновь вернемся к набору данных SST-2. Разобьем выборку на train и test аналогично заданию №6.

In [2]:
# Не меняйте этот фрагмент кода! Здесь мы скачиваем holdout-выборку,
# Т.е. это данные, на которых нам заранее неизвестна разметка. 
# Мы должны будем применить к этим данным построенную модель, и в качестве ответа предоставить сгенерированные предсказания.

# __________start of block__________

!wget https://github.com/MSUcourses/Data-Analysis-with-Python/raw/main/Deep%20Learning/holdout_texts08.npy -O holdout_texts08.npy

# __________end of block__________

--2023-04-26 17:28:03--  https://github.com/MSUcourses/Data-Analysis-with-Python/raw/main/Deep%20Learning/holdout_texts08.npy
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/MSUcourses/Data-Analysis-with-Python/main/Deep%20Learning/holdout_texts08.npy [following]
--2023-04-26 17:28:03--  https://raw.githubusercontent.com/MSUcourses/Data-Analysis-with-Python/main/Deep%20Learning/holdout_texts08.npy
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45416 (44K) [application/octet-stream]
Saving to: ‘holdout_texts08.npy’

holdout_texts08.npy 100%[===================>]  44.35K  --.-KB/s    in 0.01s   


In [3]:
# Не меняйте этот фрагмент кода! Здесь мы преобразуем holdout-выборку в numpy.array

# __________start of block__________

texts_holdout = np.load('holdout_texts08.npy', allow_pickle=True)
texts_holdout[:5]

# __________end of block__________

array(['gee , a second assassin shot kennedy ?',
       "from the big giant titles of the opening credits to elmer bernstein 's perfectly melodic score , haynes gets just about everything right",
       'the movie is well shot and very tragic , and one to ponder after the credits roll',
       'the movie quickly drags on becoming boring and predictable',
       "a mixed bag of a comedy that ca n't really be described as out of this world"],
      dtype=object)

In [4]:
# Не меняйте код ниже! Здесь мы загружаем данные с известной разметкой. На них мы можем обучать модель и вычислять метрики качества классификации.
# __________start of block__________
df = pd.read_csv(
    'https://raw.githubusercontent.com/MSUcourses/Data-Analysis-with-Python/main/Deep%20Learning/Files/STT2_train_task08.tsv',
    delimiter='\t',
    header=None
)
texts_train = df[0].values[:5000]
y_train = df[1].values[:5000]
texts_test = df[0].values[5000:]
y_test = df[1].values[5000:]

# __________end of block__________

Весь остальной код предстоит написать вам.

Для успешной сдачи на максимальный балл необходимо добиться хотя бы __84.5% accuracy на тестовой части выборки__.

In [5]:
!pip install -Uqq transformers

In [6]:
import transformers as ppb

# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (
    ppb.DistilBertModel,
    ppb.DistilBertTokenizer,
    'distilbert-base-uncased'
)

## Want BERT instead of distilBERT? Uncomment the following line:
# model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights).to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
import pandas as pd

tokenized_train = pd.Series(texts_train).apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
tokenized_test = pd.Series(texts_test).apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
tokenized_holdout = pd.Series(texts_holdout).apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))


In [8]:
max_len = 0
for i in tokenized_train.to_list() + tokenized_test.to_list() + tokenized_holdout.to_list():
    if len(i) > max_len:
        max_len = len(i)

padded_train = np.array([i + [0] * (max_len - len(i)) for i in tokenized_train.values])
padded_test = np.array([i + [0] * (max_len - len(i)) for i in tokenized_test.values])
padded_holdout = np.array([i + [0] * (max_len - len(i)) for i in tokenized_holdout.values])


In [9]:
attention_mask_train = torch.tensor(np.where(padded_train != 0, 1, 0)).to(device)
attention_mask_test = torch.tensor(np.where(padded_test != 0, 1, 0)).to(device)
attention_mask_holdout = torch.tensor(np.where(padded_holdout != 0, 1, 0)).to(device)

attention_mask_train.shape

torch.Size([5000, 67])

In [10]:
input_ids_train = torch.tensor(padded_train).to(device)
input_ids_test = torch.tensor(padded_test).to(device)
input_ids_holdout = torch.tensor(padded_holdout).to(device)

last_hidden_states_train = []

model.eval()
with torch.no_grad():
    for i in range(10):
        print(i + 1)

        idx = slice(i * 500, (i + 1) * 500)

        last_hidden_states_train.append(
            model(
                input_ids_train[idx], 
                attention_mask=attention_mask_train[idx]
            )[0][:,0,:].cpu().numpy()
        )

1
2
3
4
5
6
7
8
9
10


In [11]:
features_train = np.array(last_hidden_states_train).reshape(5000, 768)

In [12]:
last_hidden_states_test = []

model.eval()
with torch.no_grad():
    for i in range(2):
        print(i + 1)

        idx = slice(i * 1000, (i + 1) * 1000)

        last_hidden_states_test.append(
            model(
                input_ids_test[idx], 
                attention_mask=attention_mask_test[idx]
            )[0][:,0,:].cpu().numpy()
        )

1
2


In [23]:
features_test = np.vstack([last_hidden_states_test[0], last_hidden_states_test[1]])


In [15]:
features_holdout = model(input_ids_holdout, attention_mask=attention_mask_holdout)[0][:,0,:].cpu().detach().numpy()


In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier

In [42]:
from google.colab import output
output.enable_custom_widget_manager()

In [52]:
# clf = CatBoostClassifier(task_type='GPU', l2_leaf_reg=100, learning_rate=0.1, num_trees=400)
# clf.fit(features_train, y_train, eval_set=(features_test, y_test), verbose=100)

clf = LogisticRegression().fit(features_train, y_train)


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [53]:
from sklearn.metrics import accuracy_score

accuracy_score(y_train, clf.predict(features_train))

0.8824

In [54]:
accuracy_score(y_test, clf.predict(features_test))


0.8473958333333333

#### Сдача задания в контест
Сохраните в словарь `out_dict` вероятности принадлежности к нулевому и первому классу соответственно:

In [26]:
out_dict = {
    'train': clf.predict_proba(features_train),
    'test': clf.predict_proba(features_test),
    'holdout': clf.predict_proba(features_holdout),
}

Несколько `assert`'ов для проверки вашей посылки:

In [27]:
assert isinstance(out_dict['train'], np.ndarray), 'Dict values should be numpy arrays'
assert out_dict['train'].shape == (5000, 2), 'The predicted probas shape does not match the train set size'
assert np.allclose(out_dict['train'].sum(axis=1), 1.), 'Probas do not sum up to 1 for some of the objects'

assert isinstance(out_dict['test'], np.ndarray), 'Dict values should be numpy arrays'
assert out_dict['test'].shape == (1920, 2), 'The predicted probas shape does not match the test set size'
assert np.allclose(out_dict['test'].sum(axis=1), 1.), 'Probas do not sum up to 1 for some of the object'

assert isinstance(out_dict['holdout'], np.ndarray), 'Dict values should be numpy arrays'
assert out_dict['holdout'].shape == (500, 2), 'The predicted probas shape does not match the holdout set size'
assert np.allclose(out_dict['holdout'].sum(axis=1), 1.), 'Probas do not sum up to 1 for some of the object'

Запустите код ниже для генерации посылки и сдайте файл `submission_dict_hw08.npy`.

In [28]:
# do not change the code in the block below
# __________start of block__________

np.save('submission_dict_hw08.npy', out_dict, allow_pickle=True)
print('File saved to `submission_dict_hw08.npy`')
# __________end of block__________

File saved to `submission_dict_hw08.npy`


На этом задание завершено. Поздравляем!